In [2]:
import glob
import os
import multiprocessing

from itertools import repeat, product

from collections import namedtuple
from csv import writer

import torch
import torchaudio
import pandas as pd
import tqdm

import tqdm.contrib.itertools as tqdm_itertools
from IPython.display import Audio
from datasets import Audio as DSAudio, load_dataset

tqdm_chain = tqdm_itertools.itertools.chain

In [5]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_source_base = os.path.join(base_dir, 'gdsc_data/data_source')
data_processed_base = os.path.join(base_dir, 'gdsc_data/data_processed')

DATA_TAG = 'data_8c86715'

source_path = os.path.join(data_processed_base, DATA_TAG, '01_apply_ir_function/')

df = pd.read_csv(os.path.join(source_path, "metadata_grouped_train.csv"))

In [7]:
df["file_name"] = df["file_name"].map(lambda x: x.split(", ")) 

In [73]:
def worker_init(worker_exec, source_dir):
    worker_exec.source_dir = source_dir
    
def worker_main(index, variations, n_size=20):
    import random
    import shutil
    # Load initalized libraries in a process
    source_dir = worker_main.source_dir
    
    sampled_variations = random.sample(variations, n_size)
    
    for variant in set(variations).difference(set(sampled_variations)):
        var = os.path.join(source_dir, variant)
        try: 
            os.remove(var)
        except OSError:
            pass
    
    return index, sampled_variations

def worker_starmap(args):
    return worker_main(*args)

In [74]:
work_generator = ((tpl.Index, tpl.file_name) for tpl in df.itertuples())

In [75]:
n_cpu = multiprocessing.cpu_count()
source_signal_dir = os.path.join(source_path, 'train/')
pool = multiprocessing.Pool(processes=n_cpu, initializer=worker_init, initargs=(worker_main, source_signal_dir))

rows = []
for index, sampled_variations in tqdm.tqdm(pool.imap_unordered(worker_starmap, work_generator), total=df.shape[0]):
    rows.append((index, sampled_variations))

  9%|▊         | 152/1752 [05:41<59:59,  2.25s/it]  Process ForkPoolWorker-8:

Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_84/634459142.py", line 22, in worker_starmap
    return worker_main(*args)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/tmp/ipykernel_84/634

KeyboardInterrupt: 

In [70]:
index

0